In [1]:
import random
import sage.graphs.graph_plot
from sage.graphs.base.boost_graph import *
from sage.plot.point import *
import numpy

In [2]:
def seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena):
    sur_podatki = []
    for i in range(0, st_opazanj):
        mer_povezav = []
        for j in range(0, st_pov_grafa):
            mer_povezav.append(numpy.random.random_integers(min_cena, max_cena))
        sur_podatki.append(mer_povezav)
    return(sur_podatki)

#stevilo_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc

In [3]:
def mnoz(vektor1, vektor2):
    matrika = []
    for i in range(0,len(vektor1)):
        vrstica =[]
        for j in range(0, len(vektor2)):
            vrstica.append(vektor1[i] * vektor2[j])
        matrika.append(vrstica)
    return(matrika)

def kov_matrika(vse_utezi):
    avr_pov = [i * 1/len(vse_utezi) for i in [sum(x) for x in zip(*vse_utezi)]]
    #zac_mat = numpy.matrix(mnoz(numpy.subtract(vse_utezi[0],avr_pov),numpy.subtract(vse_utezi[0],avr_pov)))
    zac_mat = mnoz(numpy.subtract(vse_utezi[0],avr_pov),numpy.subtract(vse_utezi[0],avr_pov))
    for i in range(1, len(vse_utezi)):
        #zac_mat += numpy.matrix(mnoz(numpy.subtract(vse_utezi[i],avr_pov),numpy.subtract(vse_utezi[i],avr_pov)))
        zac_mat = numpy.add(zac_mat, mnoz(numpy.subtract(vse_utezi[i],avr_pov),numpy.subtract(vse_utezi[i],avr_pov)))
    kon_mat = numpy.dot(1/len(vse_utezi), zac_mat)
    return(kon_mat)

In [4]:
def vektorcki_x(graf, zacetno, koncno):
    vse_poti = graf.all_paths(zacetno, koncno, use_multiedges=True, report_edges=False, labels=False)
    vse_povezave = graf.edges()
    par = []
    for i in range(0,len(vse_povezave)):
        par.append(vse_povezave[i][0:2])
    seznam_vektorjev = []
    for i in range(0, len(vse_poti)):
        pot = (vse_poti[i])
        prvi_oklepaj1 = list(zip(pot, pot[1:] + pot[:1]))
        prvi_oklepaj = prvi_oklepaj1[:-1]
        x=[]
        for i in range(0, len(par)):
            if par[i] in prvi_oklepaj:
                x.append(1)
            else:
                x.append(0)
        seznam_vektorjev.append(x)
    return(seznam_vektorjev)

In [5]:
def grid_graf(st_opazanj, st_vozlisc, min_cena, max_cena):
    st_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc
    utezi = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    #graf_seznam = vsa_vozlisca(st_vozlisc, st_pov_grafa)
    graf_seznam = graphs.Grid2dGraph(st_vozlisc, st_vozlisc).edges()
    nov = [] #najprej spremeniva v list
    for i in range(0,len(graf_seznam)):
        nov.append(list(graf_seznam[i]))
    konec = [] #dodava še ceno, v obliki ki jo sprjema digraph
    seznam_grafov = []
    for j in range(0,len(utezi)):
        zadnji = []
        for i in range(0,len(nov)):
            vmesni = []
            vmesni.append(nov[i][0])
            vmesni.append(nov[i][1])
            vmesni.append(utezi[j][i])
            zadnji.append(tuple(vmesni))
        konec.append(zadnji)
        gr = DiGraph(zadnji)
        seznam_grafov.append(gr)
    #seznam_gra[0].show(edge_labels = True)
    return(seznam_grafov, utezi)

In [6]:
def min_grid_graf(st_opazanj, st_vozlisc, min_cena, max_cena):
    st_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc
    utezi = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    graf_oblika = DiGraph(graphs.Grid2dGraph(st_vozlisc, st_vozlisc).edges())
    return(graf_oblika, utezi)

#[grafkec, utezi] = min_grid_graf(3, 3, 1, 100)

In [7]:
def argmin(poti, av_pov, diagonala):
    x_min = poti[0]
    cena_pov = numpy.dot(av_pov, x_min)
    cena_d = numpy.dot(diagonala, x_min)
    cena = cena_pov + cena_d
    #cena = numpy.dot(numpy.add(av_pov, diagonala), poti[0])
    #if len(poti) == 1:
    #    return(x_min, cena_pov, cena_d, cena)
    #else:
    for i in range(1, len(poti)):
        x_nov = poti[i]
        cena_pov_nov = numpy.dot(av_pov, x_nov)
        cena_d_nov = numpy.dot(diagonala, x_nov)
        cena_nov = cena_pov_nov + cena_d_nov
        #cena_nov = numpy.dot(numpy.add(av_pov,diagonala), poti[i])
        if cena_nov < cena:
            x_min = x_nov
            cena_pov = cena_pov_nov
            cena_d = cena_d_nov
            cena = cena_nov
    return(x_min, cena_pov, cena_d, cena)

In [8]:
def arglexmin(seznam_x, vektorji):
    xmin = min(([numpy.dot(numpy.array(v), numpy.array(x)) for v in vektorji], x) for x in seznam_x)
    return xmin

In [9]:
def argmin_kor(poti, av_pov, diagonala):
    x_rob = poti[0]
    cena_av = numpy.dot(x_rob, av_pov)
    cena_d = sqrt(numpy.dot(x_rob, diagonala))
    cena = cena_av + cena_d
    for i in range(1, len(poti)):
        x_nov = poti[i]
        cena_av_nov = numpy.dot(poti[i], av_pov)
        cena_d_nov = sqrt(numpy.dot(poti[i], diagonala))
        cena_nov = cena_av_nov + cena_d_nov
        if cena_nov < cena:
            x_rob = x_nov
            cena = cena_nov
    return(x_rob, cena)

In [10]:
#[grafi, weights] = grid_graf(50, 9, 1, 100)
[grafi, weights] = min_grid_graf(50 , 5, 1, 100)
#g = grafi[0]
#g.show(edge_labels = True)
povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
#print(povprecje_w)
#print(kov_matrika(weights))
diagonalci = numpy.diag(kov_matrika(weights))
#print(diagonalci)
#xi = vektorcki_x(grafi[0], (0, 0), (8, 8))
xi = vektorcki_x(grafi, (0, 0), (4, 4))
#print(weights)
#print(xi)
print("dolzina xi je", len(xi))

xl = arglexmin(xi, [povprecje_w, diagonalci])
xl_vekt = xl[1]
#print("xl in njegove vr", xl)
print(xl_vekt)
xr = arglexmin(xi, [diagonalci, povprecje_w])
xr_vekt = xr[1]
#print("xr in njegove vr", xr)
print(xr_vekt)
print(xl_vekt == xr_vekt)


dolzina xi je 70
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
False


<ipython-input-2-1f0f1f17ca7f>:6: DeprecationWarning: This function is deprecated. Please call randint(1, 100 + 1) instead
  mer_povezav.append(numpy.random.random_integers(min_cena, max_cena))


In [11]:
#d = numpy.diag(kov_matrika(weights))
#average_c = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
#xl = arglexmin(xi, [average_c, d])
#xr = arglexmin(xi, [d, average_c])
#xl_vekt = xl[1]
#xr_vekt = xr[1]
prva_tocka = xl[0]
tretja_tocka = [xr[0][1], xr[0][0]]
druga_tocka = [prva_tocka[0], tretja_tocka[1]]
[x_rob, OBJ] = argmin_kor([xl_vekt, xr_vekt], povprecje_w, diagonalci)
#print(OBJ)
#najprej naredimo prvi interval ter vmesno točko m
a = prva_tocka[0]
b = OBJ - sqrt(tretja_tocka[1])
P = numpy.dot(povprecje_w, x_rob)
D = sqrt(numpy.dot(diagonalci, x_rob))

xevje = []
def intervali(rob1, rob2, OBJ, x, P, D, xi, average_c, d):
    m = (rob1 + rob2) / 2
    alfa_m = 2 * (m -OBJ) / (2 * (m - OBJ) - 1)
    [x_new, pov_x_new, diag_x_new, cena_x_new] = argmin(xi, numpy.dot(average_c, alfa_m), numpy.dot((1-alfa_m), d))
    if 2 * (m - OBJ) * P - 2 * (m - OBJ) * pov_x_new + diag_x_new <= D:
        print("x_new boljši.")
        x = x_new
        OBJ = pov_x_new + sqrt(diag_x_new)
        #OBJ = pov_x_new + diag_x_new
        P = pov_x_new
        D = sqrt(diag_x_new)
        #D = diag_x_new
        naklon = 2 * (m - OBJ)
        zac_vr = (OBJ - m)^2 - naklon * m
        var('z')
        sol = solve(0 == z^2 -(2*OBJ + naklon)*z +(OBJ^2-zac_vr), z)
        nsol = [N(s.rhs()) for s in sol]
        if nsol[0].is_real() == False or nsol[1].is_real == False:
            print("Imaginarna rešitev.")
        #    return(xevje)
        print(sol[0])
        print(sol[1])
        #print(rob1)
        if nsol[0] - rob1 > 0.001:
            print("Prvi interval dovolj velik. Nova rekurzija.")
            intervali(rob1, nsol[0], OBJ, x, P, D, xi, average_c, d)
        else:
            print("Prvi interval minimalen. Dodamo x.")
            xevje.append(x)
        if rob2 - nsol[1] > 0.001:
            print("Drugi interval dovolj velik. Nova rekurzija.")
            intervali(nsol[1], rob2, OBJ, x, P, D, xi, average_c, d)
        else:
            print("Drugi interval minimalen. Dodamo x.")
            xevje.append(x)
    else:
        print("NI boljsega x_new.")
        var('z')
        sol = solve(0 == z^2 -(2*OBJ + naklon)*z +(OBJ^2-zac_vr), z)
        nsol = [N(s.rhs()) for s in sol]
        if nsol[0].is_real() == False or nsol[1].is_real == False:
            print("Imaginarna rešitev.")
        #    return(xevje)
        if nsol[0] - rob1 > 0.001:
            intervali(rob1, nsol[0], OBJ, x, P, D, xi, average_c, d)
        else:
            xevje.append(x)
        if rob2 - nsol[1] > 0.001:
            intervali(nsol[1], rob2, OBJ, x, P, D, xi, average_c, d)
        else:
            xevje.append(x)
    return(xevje)



In [12]:
#intervali(a, b, OBJ, x_rob, P, D, xi, povprecje_w, diagonalci)

In [13]:
#improved bolša praksa
prva_tocka = xl[0]
tretja_tocka = [xr[0][1], xr[0][0]]
druga_tocka = [prva_tocka[0], tretja_tocka[1]]
[x_rob, OBJ] = argmin_kor([xl_vekt, xr_vekt], povprecje_w, diagonalci)

#najprej naredimo prvi interval ter vmesno točko m
a = prva_tocka[0]
b = OBJ - sqrt(tretja_tocka[1])
P = numpy.dot(povprecje_w, x_rob)
D = sqrt(numpy.dot(diagonalci, x_rob))

def intervali1(rob1, rob2, OBJ, x, P, D, xi, average_c, d, xevje, k):
    #xevje=[]
    m = (rob1 + rob2) / 2
    alfa_m = 2 * (m -OBJ) / (2 * (m - OBJ) - 1)
    if alfa_m >= 1:
        return(xevje)
    [x_new, pov_x_new, diag_x_new, cena_x_new] = argmin(xi, numpy.dot(average_c, alfa_m), numpy.dot((1-alfa_m), d))
    if 2 * (m - OBJ) * P - 2 * (m - OBJ) * pov_x_new + diag_x_new <= D:
        print("x_new boljši. Dodamo x.")
        x = x_new
        xevje.append(x)
        #OBJ = pov_x_new + sqrt(diag_x_new)
        #OBJ = pov_x_new + diag_x_new
        P = pov_x_new
        print("m =", m)
        print("OBJ =", OBJ)
        print("alfa_m =", alfa_m)
        print("diag=", diag_x_new)
        D = sqrt(diag_x_new)
        #D = diag_x_new
        OBJ = P + D
        naklon = 2 * (m - OBJ)
        zac_vr = (OBJ - m)^2 - naklon * m
        var('z')
        sol = solve(0 == z^2 - (2 * OBJ + naklon) * z +(OBJ^2 - zac_vr), z)
        nsol = [N(s.rhs()) for s in sol]
        #nj = nsol[0]
        if nsol[0].is_real() == True and nsol[0] - rob1 > 0.00001:
            print("Prvi interval dovolj velik. Nova rekurzija.")
            k +=1
            intervali1(rob1, nsol[0], OBJ, x, P, D, xi, average_c, d, xevje, k)
        else:
            print("Prvi interval minimalen ali ni preseka(kompleksno).")
        if nsol[1].is_real() == True and rob2 - nsol[1] > 0.00001:
            print("Drugi interval dovolj velik. Nova rekurzija.")
            k += 1
            intervali1(nsol[1], rob2, OBJ, x, P, D, xi, average_c, d, xevje , k)
        else:
            print("Drugi interval minimalen ali ni preseka(kompleksno).")
    else:
        print("NI boljsega x_new.")
        naklon = 2 * (m - OBJ)
        zac_vr = (OBJ - m)^2 - naklon * m
        var('z')
        sol = solve(0 == z^2 - (2 * OBJ + naklon) * z +(OBJ^2 - zac_vr), z)
        nsol = [N(s.rhs()) for s in sol]
        if nsol[0].is_real() == True and nsol[0] - rob1 > 0.00001:
            print("Ni izboljsanja, a vseeno iscemo naprej, 1. interval. Nova rekurzija.")
            k += 1
            intervali1(rob1, nsol[0], OBJ, x, P, D, xi, average_c, d, xevje, k)
        else:
            print("Konec1.")
            #xevje.append(x)
        if nsol[1].is_real() == True and rob2 - nsol[1] > 0.00001:
            print("Ni izboljšanja, a vseeno isčemo naprej, 2. interval. Nova rekurzija")
            k += 1
            intervali1(nsol[1], rob2, OBJ, x, P, D, xi, average_c, d, xevje, k)
        else:
            print("Konec2.")
            #xevje.append(x)
    return(xevje, k)

#intervali1(a, b, OBJ, x_rob, P, D, xi, povprecje_w, diagonalci, [])

In [0]:
intervali1(a, b, OBJ, x_rob, P, D, xi, povprecje_w, diagonalci, [], 1)